In [20]:
## incomplete
import torch
import torch.nn as nn

In [18]:
## Creating a residual block

class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, use_1x1_conv=False,
                 stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                              stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
                
        if use_1x1_conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1,
                                  stride=stride)
        else:
            self.conv3 = None
            
    def forward(self, x):
        
        y = nn.ReLU(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        
        if self.conv3:
            ## dimensions will be matched
            x = self.conv3(x)
            
        ## returns f(x) + x
        return nn.ReLU(x+y)
        
        
        



In [19]:
class ResNet(nn.Module):
    
    def __init__(self, output_size):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64,
                              kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self.get_renset_layer(64, 64, 2, first_block=True)
        self.layer2 = self.get_renset_layer(64, 128, 2)
        self.layer3 = self.get_renset_layer(128, 256, 2)
        self.layer4 = self.get_renset_layer(256, 512, 2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, output_size)
        
    
    def get_renset_layer(self, in_channels, out_channels,
                         num_residual_blocks, first_block=False):
        
        layers = list()
        
        for i in range(num_residual_blocks):
            if i == 0 and not first_block:
                layers.append(ResidualBlock(in_channels, out_channels,
                                use_1x1_conv=True, stride=2))
            else:
                layers.append(ResidualBlock(in_channels, out_channels,
                                use_1x1_conv=False, stride=1))
    
    def forward(self, x):
        x = nn.Relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        
        x = x.view(x.shape[0], -1)
        print(x.shape)
        x = self.fc(x)
        
        return x
        
        

NameError: name 'out_channels' is not defined